In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

## load all npz files from npz_files directory.

In [2]:
import os

npz_files_directory = '/home/rohan/Desktop/label-generator/raw_files/npz-files'
# collect all files from npz directory.
files_list = list()
for f in os.listdir(npz_files_directory):
    files_list.append(f)

## load all numpy arrays into training data x and y.
structure of the npz files = ['c1', 'c2', 'c3', 'Z', 'labels'].
c1, c2, c3 => numpy array containing all class 1, 2, 3 bounding boxes respectively.
Z => the raw PSD files.
labels => pixel wise labels for psd files. same dimensions as the PSD files.

In [6]:
training_data_x = list()
class_labels_x = list()
training_data_y = list()

for file_name in files_list:
    data1 = np.load(npz_files_directory + '/' + file_name)
#     print(data1.files)
#     print(file_name)
    training_data_x.append(data1['Z'])
    training_data_y.append(data1['labels'])
    # combine class labels into a single array.
    class_labels_list = [data1['c1'], data1['c2'], data1['c3']]
    labels_list = list()
    class_label = 1
    for label in class_labels_list:
        for i in range(label.shape[0]):
            labels_list.append([class_label] + list(label[i]))
            labels_list[-1] = np.array(labels_list[-1])
        class_label += 1
    class_labels_x.append(labels_list)  

In [7]:
print('shape of training_data_x', len(training_data_x))
print('shape of training_data_y', len(training_data_y))
print('shape of class_labels_x', len(class_labels_x))

shape of training_data_x 11
shape of training_data_y 11
shape of class_labels_x 11


In [11]:
print('shape of training_data_x samples', training_data_x[0].shape)
print('shape of training_data_y samples', training_data_y[0].shape)
print('shape of class_labels_x samples', class_labels_x[0][0].shape)

shape of training_data_x samples (1953, 512)
shape of training_data_y samples (1953, 512)
shape of class_labels_x samples (5,)


## variable structures:-
1. training_data_x contains all Z for all files.
2. training_data_y contains all label for all files.
3. class_label_x contains class-wise labels with format [class label, x0, y0, x1, y1] <= just for data visualizations.
4. class_label - first dimension - for each file. second dimension - for all labels in each file.

## convert training_data_x to shape (size of all timesteps, 512)

In [12]:
x = list()
y = list()

for i in range(len(training_data_x)):
    for j in range(training_data_x[i].shape[0]):
        x.append(training_data_x[i][j])
        y.append(training_data_y[i][j])

In [17]:
x = np.array(x)
y = np.array(y)

In [18]:
print('the shape of x is ', x.shape)
print('the shape of y is ', y.shape)

the shape of x is  (23436, 512)
the shape of y is  (23436, 512)


## convert into sequence data.
1. each 512 vector array is to be converted into 8 sequence timestep array.
i.e [1 2 3 4 5 6 7 8 9 10 ...] = [[1 2 3 4 5 6 7 8] [2 3 4 5 6 7 8 9] ...]
2. convert into numpy arrays.

In [19]:
def generate_sequence_data(freq_array, sequence_length):
    """
    expects a freq array of size (512, )
    returns array with sequence length pieces of freq array.
    """
    n = 512
    freq = list(freq_array)
    freq_reshaped = list()
    for k in range(n - sequence_length + 1):
        freq_reshaped.append(np.array(freq[k : k + sequence_length]))

    return np.array(freq_reshaped)

In [20]:
%%time
x_seq = list()
y_seq = list()

sequence_length = 8

for i in range(x.shape[0]):
    x_seq.append(generate_sequence_data(x[i], sequence_length))
    y_seq.append(generate_sequence_data(y[i], sequence_length))

In [23]:
print('(512, ) shaped x arrays have been reshaped to ', x_seq[0].shape)
print('(512, ) shaped y arrays have been reshaped to ', y_seq[0].shape)

(512, ) shaped x arrays have been reshaped to  (505, 8)
(512, ) shaped y arrays have been reshaped to  (505, 8)


In [24]:
print('length of freq arrays is ', len(x_seq))
print('length of label arrays is ', len(y_seq))

length of freq arrays is  23436
length of label arrays is  23436


In [25]:
x_seq = np.array(x_seq)
y_seq = np.array(y_seq)

In [27]:
print('shape of x is ', x_seq.shape)
print('shape of y is ', y_seq.shape)

shape of x is  (23436, 505, 8)
shape of y is  (23436, 505, 8)


## convert y from (23436, 505, 8) to (23436, 505, 1)

In [29]:
y_seq_cat = list()
for i in range(y_seq.shape[0]):
    y_seq_inner = list()
    for j in range(y_seq.shape[1]):
        y_seq_inner.append(np.argmax(np.bincount(y_seq[i][j])))
    y_seq_cat.append(np.array(y_seq_inner))
y_np = np.array(y_seq_cat)

In [31]:
print('shape of y is ', y_np.shape)
print('shape of y sample is ', y_np[0].shape)

shape of y is  (23436, 505)
shape of y sample is  (505,)


## reshaping x_seq, y_np

In [52]:
x_stored = x_seq
y_stored = y_np

In [59]:
x_seq_new = list()
for i in range(x_seq.shape[0]):
    x_seq_new.append(np.reshape(x_seq[i], (x_seq[i].shape[0], x_seq[i].shape[1], 1)))

In [60]:
x_seq = np.array(x_seq_new)

In [61]:
x_seq.shape

(23436, 505, 8, 1)

In [69]:
y_seq_new = list()
for i in range(y_seq.shape[0]):
    y_seq_new.append(np.reshape(y_seq[i], (y_seq[i].shape[0], y_seq[i].shape[1], 1)))

In [70]:
y_seq = np.array(y_seq_new)

In [71]:
y_seq.shape

(23436, 505, 8, 1)

In [76]:
y_seq_binary = keras.utils.to_categorical(y_seq)

In [77]:
y_seq_binary.shape

(23436, 505, 8, 3)

## Training the RNN model on the whole data.

In [79]:
rnn = keras.models.Sequential()
rnn.add(keras.layers.LSTM(32, input_shape=(8,1),  return_sequences=True))
rnn.add(keras.layers.Dense(3, activation='softmax'))

In [80]:
rnn.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adadelta(), metrics=['accuracy'])

In [81]:
for i in range(x_seq.shape[0]):
    rnn.train_on_batch(x_seq[i], y_seq_binary[i])

In [82]:
rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 8, 32)             4352      
_________________________________________________________________
dense_8 (Dense)              (None, 8, 3)              99        
Total params: 4,451
Trainable params: 4,451
Non-trainable params: 0
_________________________________________________________________


In [83]:
rnn.evaluate(x_seq[10], y_seq_binary[10])

505/505 [==============================] - 0s 397us/step


[0.15260984364800192, 0.9549504950495049]

## old code.

In [ ]:
# load data from the npz file.
training_data = np.load('processed_npz_files/data_out_psd_1074023.npz')

In [ ]:
training_data['labels'].shape

In [ ]:
np.unique(training_data['labels'], return_counts=True)

In [ ]:
x_train = training_data['Z']

In [ ]:
x_train.shape

In [ ]:
x_train[0].shape

In [ ]:
# model parameters.
sequence_length = 8

In [ ]:
y_train = training_data['labels']

In [ ]:
y = list()
for i in range(x_train.shape[0]):
    y_inner = list()
    for j in range(x_train.shape[1]-8):
        if np.count_nonzero(y_train[i, j:j+8] == 1) >= 4:
            y_inner.append(1)
        else:
            y_inner.append(0)
    y.append(np.array(y_inner, dtype=np.int8))

In [ ]:
y_np = np.array(y)

In [ ]:
y_np.shape

In [ ]:
np.unique(y_np,return_counts=True)

In [ ]:
x_train = np.reshape()

## Sequence Parameters.

In [ ]:
# model parameters.
sequence_length = 8

In [ ]:
x_train_rnn = np.reshape(x_train, (x_train.shape[0], sequence_length, 1))

In [ ]:
x_train_rnn.shape

In [ ]:
y_np = np.reshape(y_np, (y_np.shape[0], 504, 1))

In [ ]:
y_np.shape

In [ ]:
x_test = x_train[0]

In [ ]:
x_test.shape

In [ ]:
x_test[:32]

In [ ]:
x_test = np.reshape(x_test, (64, 8))

In [ ]:
x_test[0]

In [ ]:
x_test[1]

In [ ]:
print('new expected shape')
print('x dimension ', x_train.shape[0] * (x_train.shape[1] // sequence_length))

In [ ]:
x_train_rnn = np.reshape(x_train, (x_train.shape[0] * (x_train.shape[1] // sequence_length), sequence_length, 1) )

In [ ]:
y_train_rnn = np.reshape(y_train, (x_train.shape[0] * (x_train.shape[1] // sequence_length), sequence_length))

In [ ]:
x_train_rnn1 = list()
for i in range(x_train.shape[0]):
    x_train_rnn1.append(np.reshape(x_train[0], (64, 8)))
x_train_rnn1 = np.array(x_train_rnn1)

In [ ]:
x_train_rnn1.shape

In [ ]:
print('x shape', x_train.shape)
print('y shape', y_train.shape)

In [ ]:
def generate_sequence_data(index):
    n = x_train[index].shape[0]
    L = 8
    x_train_seq = []
    y_train_seq = []
    for k in range(n - L + 1):
        x_train_seq.append(x_train[index][k : k + L])
        y_train_seq.append(y_train[index][k : k + L])

    x_train_seq = np.array(x_train_seq)
    y_train_seq = np.array(y_train_seq)
    return x_train_seq, y_train_seq

In [ ]:
xy = [generate_sequence_data(i) for i in range(x_train.shape[0])]

In [ ]:
x_train_seq = list()
y_train_seq = list()
for item in xy:
    x_train_seq.append(np.reshape(item[0], (item[0].shape[0], item[0].shape[1], 1)))
    y_train_seq.append(np.reshape(item[0], (item[0].shape[0], item[0].shape[1], 1)))
    
# x_train_seq = np.array(x_train_seq)
# y_train_seq = np.array(y_train_seq)

In [ ]:
print('reshaped x shape', len(x_train_seq))
print('reshaped y shape', len(y_train_seq))

In [ ]:
x_train_seq = np.reshape(x_train_seq, (x_train_seq.shape[0], x_train_seq.shape[1], 1))
y_train_seq = np.reshape(y_train_seq, (y_train_seq.shape[0], y_train_seq.shape[1], 1))

In [ ]:
print('reshaped x shape', x_train_seq[0].shape)
print('reshaped y shape', y_train_seq[0].shape)

## Creating the RNN model.

In [ ]:
rnn = keras.models.Sequential()
rnn.add(keras.layers.LSTM(32, return_sequences=True))
rnn.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
rnn.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adadelta(), metrics=['accuracy'])

In [ ]:
for i in range(len(x_train_seq)):
    rnn.train_on_batch(x_train_seq[i], y_train_seq[i])

In [ ]:
rnn.summary()

In [ ]:
rnn.evaluate(x_train_seq[12], y_train_seq[12])